In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic
from networks.a2c_lstm.actor import ActorLSTM
from networks.a2c_lstm.critic import CriticLSTM

import gym
import matplotlib.pyplot as plt
import torch

In [2]:
norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/vinay/code/git/smart-broker/notebooks/../environment.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['rolling_price'] = self.df[self.price_typ].rolling(self.roll_period).sum()
/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [3]:
actor_model = ActorLSTM(
    state_dim=batch_dur*3+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*3+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.001),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.001),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=25,
    p_net_type='lstm',
    c_net_type='nn',
)

In [ ]:
a2c.run(5000)


Ep: 0 | L: 2.263 | R: 0.53 | P: 3.31 | R.Avg P: 3.31 | NW: 103.31 | R.Avg NW: 103.31 | R.U: 217
Ep: 25 | L: 1.002 | R: 0.54 | P: 3.39 | R.Avg P: 2.28 | NW: 103.39 | R.Avg NW: 102.28 | R.U: 203
Ep: 50 | L: 1.012 | R: 0.51 | P: -0.39 | R.Avg P: 2.8 | NW: 99.61 | R.Avg NW: 102.8 | R.U: 203
Ep: 75 | L: 1.016 | R: 0.53 | P: 1.74 | R.Avg P: 3.36 | NW: 101.74 | R.Avg NW: 103.36 | R.U: 200
Ep: 100 | L: 0.999 | R: 0.49 | P: -4.05 | R.Avg P: 2.94 | NW: 95.95 | R.Avg NW: 102.94 | R.U: 193
Ep: 125 | L: 0.992 | R: 0.51 | P: -0.44 | R.Avg P: 2.33 | NW: 99.56 | R.Avg NW: 102.33 | R.U: 186
Ep: 150 | L: 0.985 | R: 0.53 | P: 2.75 | R.Avg P: 2.74 | NW: 102.75 | R.Avg NW: 102.74 | R.U: 186
Ep: 175 | L: 0.982 | R: 0.55 | P: 4.92 | R.Avg P: 3.06 | NW: 104.92 | R.Avg NW: 103.06 | R.U: 187
Ep: 200 | L: 0.974 | R: 0.55 | P: 4.9 | R.Avg P: 2.91 | NW: 104.9 | R.Avg NW: 102.91 | R.U: 188
Ep: 225 | L: 0.965 | R: 0.52 | P: 1.14 | R.Avg P: 3.01 | NW: 101.14 | R.Avg NW: 103.01 | R.U: 192
Ep: 250 | L: 0.958 | R: 0.54

## Visualizations

### Traning 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_cl_an/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_cl_an/critic')

In [ ]:
import pickle
from datetime import datetime

temp = dict(a2c.logs)
with open(f"../pickles/a2c_logs_{datetime.now()}.pickle","wb") as f:
    pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)

### Observations

- A2C performs well when actor is chained with LSTM and critic with Neural Network
- If the actor is chained with Neural Network we dont seem to learn the task